In [47]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [48]:
epochs = 100
batch_size = 32
patience = 25
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [3]:
# Transformada de Fourier
def fft(img):
    img = np.fft.fft2(img)
    img = np.fft.fftshift(img)
    return img

# Inversa (retorna para imagem original)
def ifft(fimg):
    fimg = np.fft.ifftshift(fimg)
    fimg = np.fft.ifft2(fimg)
    return fimg

# Obtém a magnitude da imagem
def mag(img):
    absvalue = np.abs(img)
    magnitude = 20 * np.log(absvalue)
    return magnitude

# Normaliza a imagem entre 0 e 255
def norm(img):
    img = cv.normalize(
    img, None, 0, 255,
    cv.NORM_MINMAX
    )

# Melhor para ver imagens da transformada e imagens pequenas em geral.
def show(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    return img


In [49]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Conv2D(62, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(124, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(248,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(124,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 30, 30, 32)        128       
 chNormalization)                                                
                                                      

In [50]:
train = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img", 
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)


Found 3410 files belonging to 62 classes.
Using 2728 files for training.
Found 3410 files belonging to 62 classes.
Using 682 files for validation.


In [51]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100


86/86 [==============================] - 9s 93ms/step - loss: 1.1486 - accuracy: 0.6419 - val_loss: 1.7075 - val_accuracy: 0.5352
Epoch 2/100
86/86 [==============================] - 8s 94ms/step - loss: 1.1112 - accuracy: 0.6441 - val_loss: 1.5782 - val_accuracy: 0.5748
Epoch 3/100
86/86 [==============================] - 8s 94ms/step - loss: 1.0704 - accuracy: 0.6562 - val_loss: 2.6777 - val_accuracy: 0.3578
Epoch 4/100
86/86 [==============================] - 9s 98ms/step - loss: 1.1244 - accuracy: 0.6441 - val_loss: 1.6626 - val_accuracy: 0.5748
Epoch 5/100
86/86 [==============================] - 8s 92ms/step - loss: 1.1037 - accuracy: 0.6481 - val_loss: 2.4114 - val_accuracy: 0.4340
Epoch 6/100
86/86 [==============================] - 8s 94ms/step - loss: 1.0867 - accuracy: 0.6551 - val_loss: 3.3416 - val_accuracy: 0.2815
Epoch 7/100
86/86 [==============================] - 8s 95ms/step - loss: 1.0628 - accuracy: 0.6708 - val_loss: 1.6586 - val_accuracy: 0.5748
Epoch 8/100
86/86 